In [1]:
import pandas as pd

electorates = ["brindabella", "ginninderra", "kurrajong", "murrumbidgee", "yerrabi"]
final_df = pd.DataFrame()

for i, electorate in enumerate(electorates):
    data = pd.read_excel(f"https://www.elections.act.gov.au/elections_and_voting/2020_legislative_assembly_election/distribution-of-preferences-2020/table2_{electorate}.xlsx")
    quota = data.columns[list(data.columns).index("Quota = ") + 1]
    electorates[i] = [electorate, quota]

    data = pd.read_excel(f"https://www.elections.act.gov.au/elections_and_voting/2020_legislative_assembly_election/distribution-of-preferences-2020/table2_{electorate}.xlsx", header=1).T.iloc[1:]
    candidates = data[0].dropna().apply(lambda x: x.strip())
    rawParties = candidates.index
    candidates = list(candidates)
    parties = []
    party = 0
    for index, candidate in enumerate(candidates):
        if "Unnamed" not in rawParties[index]:
            party = index
        parties.append(rawParties[party])
    df = pd.DataFrame(data=parties, index=candidates, columns=["party"])
    df.index.name = "candidate"
    df.loc["exhausted"] = "nan"
    df.loc["comment"] = "nan"
    results = []
    data.set_index(0, inplace=True)
    new_index = []
    for indice in list(data.index):
        if indice == indice:
            new_index.append(indice.strip())
        else:
            new_index.append(indice)
    data.index = new_index

    for candidate in candidates + ["exhausted", "comment"]:
        array = []
        for j, column in enumerate(data.columns):
            if data.iloc[-2, j] == data.iloc[-2, j]:
                if candidate not in ["exhausted", "comment"]:
                    array.append(data.loc[candidate, column])                
                elif candidate == "exhausted":
                    array.append(data.iloc[-4, j])
                elif candidate == "comment":
                    array.append(data.iloc[-1, j])
        results.append(array)
    df["counts"] = results
    df["electorate"] = len(df) * [electorate]
    final_df = pd.concat([final_df, df])
final_df.to_csv("preferences.csv")
e_df = pd.DataFrame(electorates)
e_df.columns = ["electorate", "quota"]
e_df.to_csv("electorates.csv", index=False)